In [60]:
import pandas as pd
from time import time
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE

from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn import linear_model
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV


import lightgbm as lgb
from sklearn.metrics import precision_score, classification_report, confusion_matrix

In [34]:
runs = pd.read_csv("../input/hkracing/runs.csv")
races = pd.read_csv('../input/hkracing/races.csv')

In [35]:
runs_data = runs[['race_id', 'won', 'horse_age', 'horse_country', 'horse_type', 'horse_rating',
       'horse_gear', 'declared_weight', 'actual_weight', 'draw', 'win_odds',
       'place_odds', 'horse_id']]

In [36]:
races_data = races[['race_id', 'venue', 'config', 'surface', 'distance', 'going', 'race_class', 'date']]

In [38]:
df = pd.merge(runs_data, races_data)
df.horse_country.isnull().value_counts(ascending=True)

In [40]:
df = df.dropna()

In [41]:
df.date = pd.to_datetime(df.date)
df.date.dtype

In [17]:
X = data[['win_odds', 'draw', 'declared_weight', 'actual_weight', 'horse_rating', 
          'horse_country', 'venue', 'race_no', 'horse_ratings', 'race_class']]

In [42]:
start_time = min(df.date).strftime('%d %B %Y')
end_time = max(df.date).strftime('%d %B %Y')
no_of_horses = df.horse_id.nunique()
no_of_races = df.race_id.nunique()

print(f'The dataset was collected from {start_time} to {end_time}, which contains information about {no_of_horses} horses and {no_of_races} races. ')

In [43]:
df = df.drop(columns=['horse_id', 'date'])

In [44]:
def horse_gear_impute(cols):
    if cols == '--':
        return 0
    else: 
        return 1

In [45]:
df.horse_gear = df.horse_gear.apply(horse_gear_impute)

In [47]:
df = pd.get_dummies(df, drop_first=True)

In [51]:
last_raceid = max(df.race_id)
last_race = df[df.race_id == last_raceid]

In [52]:
new_data = df[:75696]   # drop the last race data for modeling
new_data = new_data.drop(columns='race_id')   # drop the unnecessary race_id column

In [53]:
X = new_data.drop(columns='won')
y = new_data['won']

In [54]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4)

In [55]:
k_range = range(1,10)
scores = {}
scores_list = []

for k in k_range:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    # precision ratio: tp / (tp + fp), aiming at minimize fp (predict: win, actual: lose)
    scores[k] = precision_score(y_test, y_pred)
    scores_list.append(precision_score(y_test, y_pred))

In [57]:
import operator
max(scores.items(), key=operator.itemgetter(1))

In [58]:
mnb = MultinomialNB()
scores = cross_val_score(mnb, X_train, y_train, cv=10, scoring='precision')
average_precision = sum(scores) / len(scores) 
print(f'MultinomialNB average precision: {average_precision}')

In [61]:
lr= linear_model.LinearRegression()
cv= cross_val_score(lr,X_train,y_train,cv=5)
print(cv)
print(cv.mean())

In [62]:
knn= KNeighborsRegressor(n_neighbors=4)
cv= cross_val_score(knn,X_train,y_train,cv=5)
print(cv)
print(cv.mean())

In [63]:
tree=DecisionTreeRegressor(random_state=1)
cv= cross_val_score(tree,X_train,y_train,cv=5)
print(cv)
print(cv.mean())

In [64]:
rf=RandomForestRegressor(random_state=1)
cv= cross_val_score(rf,X_train,y_train,cv=5)
print(cv)
print(cv.mean())

In [67]:
lr.fit(X_train,y_train)
y_lr=lr.predict(X_test)
#"---------------------------------------------------------------------------------------------------")
rf.fit(X_train,y_train)
y_rf=rf.predict(X_test)

In [68]:
print("Linear Regression results:")
print("R^2",metrics.r2_score(y_test,y_lr))
print("Mean Absolute Error", metrics.mean_absolute_error(y_test,y_lr))
print("Mean Squared Error", metrics.mean_squared_error(y_test,y_lr))
print("Root Mean Squared Error",np.square(metrics.mean_squared_error(y_test,y_lr)))

print("---------------------------------------------------------------------------------------------------")

print("Random Forest results:")
print("R^2",metrics.r2_score(y_test,y_rf))
print("Mean Absolute Error", metrics.mean_absolute_error(y_test,y_rf))
print("Mean Squared Error", metrics.mean_squared_error(y_test,y_rf))
print("Root Mean Squared Error",np.square(metrics.mean_squared_error(y_test,y_rf)))